# Overview
In this notebook we will run a simple illustration on how to 
1. Import workflow configuration into your Cyoda environment
2. Register entity models with Cyoda so that you can save entities
3. Save an entity for a given model
4. Run a search and retrieve it's results

## Nobel Prizes dataset
We will use two models related to Nobel Prize data. The first is a dataset structure containing a list of Nobel Prizes. The second is a single Nobel Prize for a given category and year. The workflow for the dataset is configured to call a Processor that dissects the data set and saves each Nobel Prize in the set as single Nobel Prize entity. See the Kotlin code for more details. 

Basically, when the dataset is saved, its workflow will trigger the processor and create hundreds of single prizes.

## Prerequites
1. You need to have your username and password ready.
2. Your Cyoda environment must be up and running.
3. The Spring Boot `SimpleExampleCyodaClient` application that connects to Cyoda as a compute node has to be running.

# Install required libs

In [ ]:
import sys
!{sys.executable} -m pip install tzlocal

# Define your connection parameters and credentials
## Set your namespace

In [20]:
namespace = 'put your Cyoda namespace here'
api_url = f"https://{namespace}.cyoda.net/api"

# If you have a Cyoda Platform license, you might have Cyoda running locally on your laptop. 
#api_url = 'http://localhost:8082/api'

login_endpoint = f"{api_url}/auth/login"
token_endpoint = f"{api_url}/auth/token"

## Set your credentials
To avoid being prompts for a password to connect, you can set an environment variable `DEMO_USER_PASSWD`, for example in your personal github Codespace Secrets.

In [21]:
from pathlib import Path
import getpass
import os

default_username = 'demo.user'

username = default_username
password = os.getenv('DEMO_USER_PASSWD')

# If the environment variable is not set, check the password file (only relevant when running this book locally)
if not password:
    password_file = Path('/Users/paul/.cyoda/demo.passwd')

    if password_file.exists():
        # Read the password from the file
        with password_file.open('r') as file:
            password = file.read().rstrip()
    else:
        # Prompt for credentials when no env variable or file is available
        password = getpass.getpass("Enter your password: ")

credentials = {
    'username': username,
    'password': password
}

# Import your workflow configuration
Import everying from the config directory using the `cyoda_config_ctl.py` script

In [ ]:
target_dir = 'src/main/resources/cyoda/config/nobel-prizes/cyoda-config'

!{sys.executable} src/tools/cyoda_config_ctl.py \
    -m 'import' \
    -host "{api_url}" \
    -u "{username}" \
    -pw "{password}" \
    -fd "{target_dir}" \
    --need_to_import_state_machine true

# Some Functions
To do this work, we need to do a bunch of HTTP API requests to your Cyoda environment.

I'm a python noob, so I'm just going to write lots of old school functions that interact with Cyoda

In [23]:
import requests
import json
import time
from datetime import datetime
import tzlocal  # For detecting local timezone

def login_and_get_refresh_token(credentials):

    headers = {
        'X-Requested-With': 'XMLHttpRequest',
        'Content-Type': 'application/json'
    }
    
    payload = json.dumps(credentials)
    
    response = requests.post(login_endpoint, headers=headers, data=payload)

    if response.status_code == 200:
        # Assuming the refresh token is returned in the 'refresh_token' field
        refresh_token = response.json().get('refreshToken')
        return refresh_token
    else:
        raise Exception(f"Login failed: {response.status_code} {response.text}")
    
#############################################################################
# Get an access token from the refresh token
#############################################################################
def get_access_token(refresh_token):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {refresh_token}'
    }
    response = requests.get(token_endpoint, headers=headers)

    if response.status_code == 200:
        token_data = response.json()
        access_token = token_data.get('token')
        #token_expiry = token_data.get('tokenExpiry')
        return access_token
    else:
        raise Exception(f"Token refresh failed: {response.status_code} {response.text}")

#############################################################################
# Check if the given model exists
#############################################################################
def model_exists(model_name,model_version):
    export_model_url = f"{api_url}/treeNode/model/export/SIMPLE_VIEW/{model_name}/{model_version}"
    
    response = requests.get(export_model_url, headers=headers)
    
    if response.status_code == 200:
        return True
    elif response.status_code == 404:
        return False
    else:
        raise Exception(f"Get: {response.status_code} {response.text}")


#############################################################################
# Get the definition of a model
#############################################################################
def get_model(model_name,model_version):
    export_model_url = f"{api_url}/treeNode/model/export/SIMPLE_VIEW/{model_name}/{model_version}"
    
    response = requests.get(export_model_url,headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Getting the model failed: {response.status_code} {response.text}")


#############################################################################
# Get the state of the model, i.er. LOCKED or UNLOCKED
#############################################################################
def get_model_state(model_name,model_version):
    export_model_url = f"{api_url}/treeNode/model/export/SIMPLE_VIEW/{model_name}/{model_version}"
    
    response = requests.get(export_model_url,headers=headers)
    
    if response.status_code == 200:
        return response.json().get('currentState')            
    else:
        raise Exception(f"Failed to get the model: {response.status_code} {response.text}")  


#############################################################################
# Unlock a model. Will only succeed if there is no data for that model
#############################################################################
def unlock_model(model_name,model_version):
    unlock_model_url = f"{api_url}/treeNode/model/{model_name}/{model_version}/unlock"

    response = requests.put(unlock_model_url,headers=headers)
    if response.status_code == 200:
        print('model unlocked')
    else:
        raise Exception(f"Unlock failed: {response.status_code} {response.text}")


#############################################################################
# Lock a model. You cannot save data for a model until it is LOCKED
#############################################################################
def lock_model(model_name,model_version):
    lock_model_url = f"{api_url}/treeNode/model/{model_name}/{model_version}/lock"

    response = requests.put(lock_model_url,headers=headers)
    if response.status_code == 200:
        print('model locked')
    else:
        raise Exception(f"Lock failed: {response.status_code} {response.text}")


#############################################################################
# Delete a model. You can only delete a model that is UNLOCKED
#############################################################################
def delete_model(model_name,model_version):
    model_url               = f"{api_url}/treeNode/model/{model_name}/{model_version}"
    
    response = requests.delete(model_url,headers=headers)
    if response.status_code == 200:
        print('model deleted')
    else:
        raise Exception(f"Deletion of the model failed: {response.status_code} {response.text}")


def calculate_total_entities_removed(json_string):
    # Parse the JSON string into a Python list
    data = json.loads(json_string)
    
    # Initialize the total counter
    total_entities_removed = 0
    
    # Iterate through each entry and accumulate the number of removed entities
    for entry in data:
        total_entities_removed += entry['deleteResult']['numberOfEntititesRemoved']
    
    return total_entities_removed


#############################################################################
# Delete all data for a given model
#############################################################################
def delete_all_entities(model_name,model_version):
    delete_entities_url = f"{api_url}/entity/TREE/{model_name}/{model_version}"

    params = {
        'pageSize': '1000',
        'transactionSize': '1000'
    }
    response = requests.delete(delete_entities_url, headers=headers, params=params)

    if response.status_code == 200:
    
        # Initialize the total counter
        total_entities_removed = 0
    
        # Iterate through each entry and accumulate the number of removed entities
        for entry in response.json():
            total_entities_removed += entry['deleteResult']['numberOfEntititesRemoved']
        return total_entities_removed
    
    else:
        raise Exception(f"Deletion failed: {response.status_code} {response.text}")


#############################################################################
# Specify a model via sample data
#############################################################################
def derive_model_from_sample_data(model_name,model_version,payload):
    import_model_url = f"{api_url}/treeNode/model/import/JSON/SAMPLE_DATA/{model_name}/{model_version}"
    response = requests.post(import_model_url, headers=headers, data=payload)
    if response.status_code == 200:
        return response.text
    else:
        raise Exception(f"Save failed: {response.status_code} {response.text}")
        
#############################################################################
# Reset a model, i.e.
#   - check if it exists, and if so delete all data for that model, then
#        unlock it and delete it
#   - (re)create the model from the sample data of the given file and lock it
#############################################################################
def reset_model(model_name, model_version, file_path):
    print(f"Resetting model '{model_name}' version {model_version}")
    
    is_an_existing_model = model_exists(model_name,model_version)
    
    if is_an_existing_model: 
        print(f"Deleting all data for model '{model_name}' version {model_version}")
        delete_result = delete_all_entities(model_name,model_version)
        print(f"Total entities deleted: {delete_result}")
        
        current_model_state = get_model_state(model_name,model_version)
        
        if current_model_state == 'LOCKED':
            unlock_model(model_name,model_version)
        else:
            print('Model not locked')
    
        delete_model(model_name,model_version)
    else:
        print(f"model {model_name} {model_version} doesn't exist. Nothing to do")

    with open(file_path, 'r') as file:
         file_contents = json.load(file)
    
    payload = json.dumps(file_contents)
    
    model_id = derive_model_from_sample_data(model_name,model_version,payload)
    
    print(f"model id = {model_id}")

    lock_model(model_name,model_version)
    print('Model locked')

#############################################################################
# Save a new entity for the given JSON representation
#############################################################################
def create_entity(model_name,model_version,json_payload):
    create_entity_url = f"{api_url}/entity/JSON/TREE/{model_name}/{model_version}"

    params = {
        'transactionTimeoutMillis': '10000'
    }

    response = requests.post(create_entity_url, headers=headers, params=params, data=json_payload)

    if response.status_code == 200:
        json_response = response.json()
        print(response.text)
        
        # Assert that there is only one transaction
        assert len(json_response) == 1, f"Expected 1 transaction, but got {len(json_response)}"
        
        transaction = json_response[0]
        entity_ids = transaction['entityIds']
        
        # Assert that the list of entityIds has exactly one element
        assert len(entity_ids) == 1, f"Expected 1 entityId, but got {len(entity_ids)}"
        
        return entity_ids[0]
    else:
        raise Exception(f"Save failed: {response.status_code} {response.text}")

#############################################################################
# Get entities for a given model. This is paged, so you need to provide 
# the page size and page number. 
#
# We still need to provide in the Cyoda backend the statistics endpoints 
# for model data and also enhance this endpoint to provide HATEOAS links 
# to make it easier to page through all data. It's a work-in-progress
#############################################################################
def get_all_entities(model_name,model_version,page_size,page_number):
    delete_entities_url = f"{api_url}/entity/TREE/{model_name}/{model_version}"

    params = {
        'pageSize': f"{page_size}",
        'pageNumber': f"{page_number}"
    }

    response = requests.get(delete_entities_url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Get all entities failed: {response.status_code} {response.text}")

#############################################################################
# Trigger a search for entities for the given condition 
#############################################################################
def create_snapshot_search(model_name,model_version,condition):
    search_url = f"{api_url}/treeNode/search/snapshot/{model_name}/{model_version}"

    response = requests.post(search_url, headers=headers, data=json.dumps(condition))
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Snapshot search trigger failed: {response.status_code} {response.text}")

#############################################################################
# Check the status of the snapshot search
#############################################################################
def get_snapshot_status(snapshot_id):
    status_url = f"{api_url}/treeNode/search/snapshot/{snapshot_id}/status"

    response = requests.get(status_url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Snapshot search trigger failed: {response.status_code} {response.text}")


#############################################################################
# Wait for the completion of the snapshot search until a timeout is reached
#############################################################################
def wait_for_search_completion(snapshot_id, timeout=5, interval=10):
    """
    Waits until the status is 'completed' or 'failed', or the timeout is exceeded.
    
    Parameters:
    - timeout: Max time to wait in seconds.
    - interval: Time to wait between status checks in msec.
    
    Returns:
    - snapshot status response if the task finishes successfully.
    - Raises an Exception if the timeout is exceeded or the task fails.
    """
    start_time = time.time()  # Record the start time
    
    while True:
        status_response = get_snapshot_status(snapshot_id)
        status = status_response.get("snapshotStatus")
        
        # Check if the status is SUCCESSFUL or FAILED
        if status == "SUCCESSFUL":
            return status_response
        elif status != "RUNNING":
            raise Exception(f"Snapshot search failed: {json.dumps(status_response, indent=4)}")
        
        elapsed_time = time.time() - start_time
        
        if elapsed_time > timeout:
            raise TimeoutError(f"Timeout exceeded after {timeout} seconds")
        
        time.sleep(interval/1000)  # Wait for the given interval (msec) before checking again

def get_search_result(snapshot_id,page_size,page_number):
    result_url = f"{api_url}/treeNode/search/snapshot/{snapshot_id}"

    params = {
        'pageSize': f"{page_size}",
        'pageNumber': f"{page_number}"
    }

    response = requests.get(result_url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Get search result failed: {response.status_code} {response.text}")


#############################################################################
# Search for entities
#############################################################################    
def search_entities(model_name,model_version,condition):
    snapshot_id = create_snapshot_search(model_name,model_version,condition)
    status_response = wait_for_search_completion(snapshot_id)
    status_response['snapshotId'] = snapshot_id
    return status_response

#############################################################################
# For pretty printing of an ISO formatted datetime
#############################################################################    
def convert_to_local_time(iso_string):
    """
    Convert an ISO 8601 formatted datetime string to the local timezone.

    :param iso_string: A string representing the datetime in ISO 8601 format
    :return: A string with the datetime in the local timezone, formatted as 'YYYY-MM-DD HH:MM:SS TZ (offset)'
    """
    # Parse the ISO 8601 string
    parsed_date = datetime.fromisoformat(iso_string)

    # Detect the local timezone dynamically
    local_timezone = tzlocal.get_localzone()

    # Convert the parsed datetime to the detected local timezone
    local_date = parsed_date.astimezone(local_timezone)

    # Format the datetime in a human-readable form with the timezone
    return local_date.strftime('%Y-%m-%d %H:%M:%S %Z (%z)')


# Reset the data models
We'll be using two models: a collection of Nobel Prizes and an individual Nobel Prize.
In workflow, the collection will be dissected and for each an individual Nobel Prize saved.

To reset a model, we first check if the model exists and if so delete all data for that model, then unlock it and delete the model itself. We can do this, because it's a sandbox here. In real life, we would need to open up a new model version and either live with multple versions, or migrate data from one version to the next.

Once it's deleted, we'll create it again, using the appropriate sample data. Before saving any entities, we have to lock the model, so that it is immutable, because a model must always be consistent with its entities. If you want to change a model after you have saved data for it, you must open up a new version for that model.

The paradigm is that models are immutable when data is attached.

In [ ]:
refresh_token = login_and_get_refresh_token(credentials=credentials)
access_token = get_access_token(refresh_token=refresh_token)
headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }

prizes_model = 'prizes'
prizes_sample = './src/main/resources/cyoda/config/nobel-prizes/sample-data/prize-physics-2019.json'

single_prize_model = 'prize'
single_prize_sample = './src/main/resources/cyoda/config/nobel-prizes/sample-data/single-prize-physics-2019.json'

model_version = 1

print("WARNING: Deleting lots of entities is still slow. Be patient and wait for the DONE signal")

reset_model(prizes_model,model_version,prizes_sample)
reset_model(single_prize_model,model_version,single_prize_sample)

print('DONE with reset')

# Post the Nobel Prizes full dataset
We now can post the full Nobel Prize dataset. It'll be a large entity and is fully searchable. But for illustration purposes we will use workflow to dissect each item in the dataset to create a new entity for each Nobel Prize.

In [ ]:
# This dataset in only a partial list of all prizes
file_path = './src/main/resources/cyoda/config/nobel-prizes/sample-data/prize.json'

with open(file_path, 'r') as file:
    file_contents = json.load(file)
    
json_payload = json.dumps(file_contents)
entity_id = create_entity(prizes_model,model_version,json_payload) 
print(f"Entity Id = {entity_id}")

# Check result
See if we have created individual Nobel Prizes entities in workflow. 

We will use the snapshot search API. This runs a snapshot search for the given condition, which will use indexing wherever possible. The search is done with horizontal scalability, meaning that each of the Cyoda nodes for your environment will do a part of the work (i.e. the search is sharded across the cluster). If your queries involve full table scans, you can linearly increase the performance by adding more Cyoda nodes to your environment. The query is resillient against node failures, by the way. The cluster redistributes the work automatically. Unless something really bad happens, your query will always return a result, even if nodes fail while the query is running.

After launching the query, check the status until the report completes or a timeout is reached. Once complete, print the first page.

Snapshot search results are available for a certain amount of time and then are automatically deleted. You can find out how long the results are available from the search status request.

In [ ]:
condition = {
    "operator": "AND",
    "conditions": [
        {
            "jsonPath": "$.dataSetId",
            "operatorType": "EQUALS",
            "value": f"{entity_id}",
            "type": "simple"
        }
    ],
    "type": "group"
}

search_status = search_entities(single_prize_model,model_version,condition)

print(f"Found {search_status.get("entitiesCount")} entities")
expiration_date = convert_to_local_time(search_status.get("expirationDate"))
print(f"Snapshot will expire on {expiration_date}")

snapshot_id = search_status['snapshotId']
first_page = get_search_result(snapshot_id,page_size=10,page_number=1)

print(json.dumps(first_page, indent=4))